# SPLF Offline Backtest — Project Overview

This notebook guides you through running the SPLF backtest pipeline (download → 1m bars → features → backtest → analysis) using the helper class `SPLFNotebook`.

## 0) Setup
- Ensure you installed deps: `pip install -r requirements.txt` (run in your terminal or uncomment the cell below).
- Configure your symbols/date range in `config/config.yaml` (this notebook will create it from the example if missing).

In [ ]:
# Optional: install dependencies in this environment
# !pip install -r requirements.txt
import sys, platform
print('Python:', sys.version.split()[0], '| Platform:', platform.platform())

In [ ]:
from pathlib import Path
import shutil
from splf.notebook import SPLFNotebook

CONFIG_PATH = 'config/config.yaml'
if not Path(CONFIG_PATH).exists():
    Path('config').mkdir(parents=True, exist_ok=True)
    shutil.copy('config/example.config.yaml', CONFIG_PATH)

nb = SPLFNotebook(CONFIG_PATH)
print('Using config:', CONFIG_PATH)
print('Symbols:', nb.symbols)
print('Period:', nb.cfg['period']['start'], '→', nb.cfg['period']['end'])

## 1) (Optional) Download raw data
This step pulls daily zip files from Binance Vision (can be large and slow). You can skip if you already have data in `data/raw/`.

In [ ]:
# WARNING: large downloads — uncomment to run
# df_dl = nb.download()
# df_dl.head()

## 2) Build 1-minute bars
Parses raw zips, normalizes timestamps, aligns into a 1-minute grid per symbol.

In [ ]:
from IPython.display import display
minute = nb.build_minute(return_df=True)
sym = nb.symbols[0] if nb.symbols else None
print('Built symbols:', list(minute.keys()))
if sym and minute.get(sym) is not None and not minute[sym].empty:
    display(minute[sym].head())
else:
    print('No 1m data to display — check raw data or config period.')

## 3) Compute features (5m, refreshed per 1m)
Computes basis/funding proxies, CVD, spread, RV, etc., then downsamples to 5-minute bars.

In [ ]:
feats = nb.features(return_df=True)
if sym and feats.get(sym) is not None and not feats[sym].empty:
    display(feats[sym].head())
else:
    print('No features to display — check previous step.')

## 4) Run walk-forward backtest
Fits Isolation Forest on a rolling 30d window, re-trains every 8h, thresholds via recent quantile, applies persistence (pre-alert + confirm).

In [ ]:
alerts_map = nb.backtest()
alerts = alerts_map.get(sym)
if alerts is not None and not alerts.empty:
    display(alerts.head())
else:
    print('No alerts produced — try longer period or adjust config.')

## 5) Analyze outcomes & metrics
Attaches explosion labels by horizon and computes precision/recall/F1.

In [ ]:
metrics, outcomes = nb.analyze()
print(metrics)
display(outcomes.head() if outcomes is not None and not outcomes.empty else 'No outcomes to show')

## Next steps
- Tweak `config/config.yaml` (symbols, date range, thresholds).
- Expand to Tier-B/C universes, enable spot aggTrades for BTC/ETH.
- Add visualizations (PR curves, lead-time histograms).